In [163]:
import os
import re
import sqlite3

import numpy as np
import pandas as pd
from datasets import load_dataset, load_from_disk
from nltk.corpus import stopwords
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

stop_words = list(stopwords.words("english"))
print(len(stop_words))
stop_words2 = text.ENGLISH_STOP_WORDS
stop_words = list(stop_words2.union(stop_words))

print(len(stop_words))

179
378


In [164]:
# 読み込み
df = pd.read_csv("../preprocessed/334_tfidf_gpu/000/data2.csv")
file_path = "../dataset/llm-science-fts/fts.db"
db = sqlite3.connect(file_path)
cur = db.cursor()

In [174]:
cur.execute("PRAGMA synchronous = OFF")
cur.execute("PRAGMA journal_mode = OFF")
cur.execute("PRAGMA locking_mode = EXCLUSIVE")
cur.execute("PRAGMA count_changes = OFF")
"""
PRAGMA temp_store = MEMORY;
PRAGMA PAGE_SIZE = 4096;
PRAGMA default_cache_size=700000; 
PRAGMA cache_size=700000;
"""
db.commit()

In [175]:
%%time

index = 806
text = (
    df.iloc[index].prompt
    + " "
    + df.iloc[index].A
    + " "
    + df.iloc[index].B
    + " "
    + df.iloc[index].C
    + " "
    + df.iloc[index].D
    + " "
    + df.iloc[index].E
)

print("【Prompt】 ", df.iloc[index]["prompt"])
answer = df.iloc[index]["answer"]
print(f"【Answer {answer}】 ", df.iloc[index][df.iloc[index]["answer"]])
two_vectorizer = CountVectorizer(analyzer="word", ngram_range=(2, 2), stop_words=stop_words, max_features=20)
tfidf_matrix = vectorizer.fit_transform([text])
features = list(vectorizer.get_feature_names_out())
q = " OR ".join(["(" + " AND ".join(text.split()) + ")" for text in features])
print(q)


res = cur.execute(
    f"""select text, rank
      from imdb
      where text MATCH "{q}"
      ORDER BY rank
      limit 10""",
).fetchall()


res

【Prompt】  What is the cause of abnormal calcification/bone formation in the condition known as Diffuse idiopathic skeletal hyperostosis (DISH)?
【Answer B】  The natural aging process, causing degeneration of the ligaments and subsequent bone formation.
(abnormal) OR (bone) OR (formation) OR (joints) OR (known) OR (leading) OR (ligaments) OR (longitudinal) OR (mechanism) OR (natural) OR (nerves) OR (presence) OR (protective) OR (response) OR (skeletal) OR (soft) OR (spinal) OR (spine) OR (surrounding) OR (tissues)
CPU times: user 17.5 s, sys: 898 ms, total: 18.4 s
Wall time: 18.4 s


[('Diffuse idiopathic skeletal hyperostosis > Diffuse idiopathic skeletal hyperostosis (DISH) is a condition characterized by abnormal calcification/bone formation (hyperostosis) of the soft tissues surrounding the joints of the spine, and also of the peripheral or appendicular skeleton. In the spine, there is bone formation along the anterior longitudinal ligament and sometimes the posterior longitudinal ligament, which may lead to partial or complete fusion of adjacent vertebrae. The facet and sacroiliac joints tend to be uninvolved. The thoracic spine is the most common level involved. In the peripheral skeleton, DISH manifests as a calcific enthesopathy, with pathologic bone formation at sites where ligaments and tendons attach to bone. The majority of people with DISH are not symptomatic, and the findings are an incidental imaging abnormality. In some, the x-ray findings may correspond to symptoms of back stiffness with flexion/extension or with mild back pain. Back pain or stiffn

In [102]:
%%time

index = 509
text = (
    df.iloc[index].prompt
    + " "
    + df.iloc[index].A
    + " "
    + df.iloc[index].B
    + " "
    + df.iloc[index].C
    + " "
    + df.iloc[index].D
    + " "
    + df.iloc[index].E
)
print(df.iloc[index])
vectorizer = TfidfVectorizer(stop_words=stop_words, max_features=20)
tfidf_matrix = vectorizer.fit_transform([text])
features = vectorizer.get_feature_names_out()
print(features)

q = " OR ".join(features)

res = cur.execute(
    f"""select text, rank
      from imdb
      where text MATCH "{q}"
      ORDER BY rank
      limit 10""",
).fetchall()

print(q)

res

prompt     What is the approximate distance of NGC 7129 f...
A                                             33 light years
B                                                3,300 miles
C                                           3,300 kilometers
D                                            3.3 light years
E                                            330 light years
answer                                                     D
context    NGC 2623 > Environment > NGC 2623 is located i...
Name: 509, dtype: object
['300' '33' '330' '7129' 'approximate' 'distance' 'earth' 'kilometers'
 'light' 'miles' 'ngc' 'years']
300 OR 33 OR 330 OR 7129 OR approximate OR distance OR earth OR kilometers OR light OR miles OR ngc OR years
CPU times: user 16.9 s, sys: 868 ms, total: 17.8 s
Wall time: 17.8 s


[('Interstellar travel > As of January 20, 2023, Voyager 1, the farthest human-made object from Earth, is 159 AU away. The closest known star, Proxima Centauri, is approximately 268,332 AU away, or over 9,000 times farther away than Neptune. Because of this, distances between stars are usually expressed in light-years (defined as the distance that light travels in vacuum in one Julian year) or in parsecs (one parsec is 3.26 ly, the distance at which stellar parallax is exactly one arcsecond, hence the name). Light in a vacuum travels around 300,000 kilometres (186,000 mi) per second, so 1 light-year is about 9.461×1012 kilometers (5.879 trillion miles) or 63,241 AU. Hence, Proxima Centauri is approximately 4.243 light-years from Earth. Another way of understanding the vastness of interstellar distances is by scaling: One of the closest stars to the Sun, Alpha Centauri A (a Sun-like star that is one of two companions of Proxima Centauri), can be pictured by scaling down the Earth–Sun di

In [3]:
wiki_df = pd.read_parquet("../input/llm-science-wikipedia-data-b/n.parquet")

In [4]:
wiki_df.head()

,id,title,text
0,59737058,Novi Radio Bihać,Novi Radio Bihać or Novi Radio is a Bosnian lo...
1,59739362,Nathan Bastian,"Nathan Bastian (born December 6, 1997) is a Ca..."
2,5973939,NATO (disambiguation),"Look up NATO, Nato, nato, or nǟtõ in Wiktionar..."
3,59739520,Nykøbing Falster Idrætspark,Nykøbing Falster Idrætspark is a football stad...
4,59739655,Nikolai Dimitrievich Dabić,Nikolai Dimitrievich Dabić (also spelled Dabit...


In [5]:
%%time
file_path = ":memory:"

if os.path.exists(file_path):
    # ファイルを削除
    os.remove(file_path)
db = sqlite3.connect(file_path)
cur = db.cursor()
cur.execute('create virtual table imdb using fts5(text, tokenize="porter unicode61");')  # UNINDEXED

CPU times: user 2.21 ms, sys: 702 µs, total: 2.91 ms
Wall time: 1.41 ms


In [6]:
%%time
cur.executemany(
    "insert into imdb (text) values (?);",
    wiki_df[["text"]].to_records(index=False),
)
db.commit()

CPU times: user 58.2 s, sys: 1.28 s, total: 59.5 s
Wall time: 59.5 s


In [95]:
index = 668
text = (
    df.iloc[index].prompt
    + " "
    + df.iloc[index].A
    + " "
    + df.iloc[index].B
    + " "
    + df.iloc[index].C
    + " "
    + df.iloc[index].D
    + " "
    + df.iloc[index].E
)
text = re.sub(r"[^a-zA-Z0-9]", " ", text)
filtered_words = [word for word in text.split() if word.lower() not in stop_words]
print(df.iloc[index])
print(filtered_words)

prompt      Which term best describes the structural organ...
A           Lactose permease is the rarely used term that ...
B           Lactose permease is the study of the relations...
C           Lactose permease is the study of the effects o...
D           Lactose permease is a branch of morphology tha...
E           Lactose permease is the analysis of the patter...
answer                                                      B
context     Lactose permease > Lactose permease is a membr...
sim_max                                              0.226259
sim_min                                              0.159622
sim_mean                                              0.20019
sim_std                                              0.015267
sim_num                                                    19
Name: 668, dtype: object
['term', 'best', 'describes', 'structural', 'organization', 'lactose', 'permease', 'Lactose', 'permease', 'rarely', 'used', 'term', 'refers', 'superstructure', 'polymers

In [13]:
%%time

q = " OR ".join(filtered_words)

res = cur.execute(
    f"""select text
                      from imdb
                      where text MATCH "{q}"
                      ORDER BY rank
                      limit 2""",
).fetchall()

print()
print(q)

res


approximate OR distance OR NGC OR 7129 OR Earth OR 33 OR light OR years OR 3 OR 300 OR miles OR 3 OR 300 OR kilometers OR 3 OR 3 OR light OR years OR 330 OR light OR years
CPU times: user 522 ms, sys: 12 ms, total: 534 ms
Wall time: 532 ms


[("NGC 7129 is a reflection nebula located 3,300 light years away in the constellation Cepheus. A young open cluster is responsible for illuminating the surrounding nebula. A recent survey indicates the cluster contains more than 130 stars less than 1 million years old.[citation needed] NGC 7129 is located just half a degree from nearby cluster NGC 7142. The nebula is rosebud-shaped; the young stars have blown a large, oddly shaped bubble in the molecular cloud that once surrounded them at their birth. The rosy pink color comes from glowing dust grains on the surface of the bubble being heated by the intense light from the young stars within. The ultra-violet and visible light produced by the young stars is absorbed by the surrounding dust grains. They are heated by this process and release the energy at longer infrared wavelengths as photographed by the Spitzer Space Telescope. The reddish colors in the false-colour infrared image suggest the distribution of hydrocarbon rich molecular

In [12]:
df = pd.read_csv("../preprocessed/331_retrieve_b/b_bge_10_4_3/data2.csv")
preds = np.load("../output/350_1st_infer/b_bge_10_4_3/data2_pred.npy")


option_to_index = {option: idx for idx, option in enumerate("ABCDE")}
index_to_option = {v: k for k, v in option_to_index.items()}


def predictions_to_map_output_top(predictions):
    sorted_answer_indices = np.argsort(-predictions)  # Sortting indices in descending order
    top_answer_indices = sorted_answer_indices[:, :]  # Taking the first three indices for each row
    top_answers = np.vectorize(index_to_option.get)(
        top_answer_indices
    )  # Transforming indices to options - i.e., 0 --> A
    return np.apply_along_axis(lambda row: " ".join(row[0]), 1, top_answers)


valid_preds = predictions_to_map_output_top(preds[: len(df), :])
df["preds"] = valid_preds

df[df["preds"] != df["answer"]].index

Int64Index([  32,   39,   65,   68,   96,  120,  124,  135,  147,  178,
            ...
            1148, 1157, 1159, 1165, 1180, 1182, 1183, 1185, 1188, 1194],
           dtype='int64', length=202)

In [2]:
paraphs_parsed_dataset = load_from_disk("../input/all-paraphs-parsed-expanded")
modified_texts = paraphs_parsed_dataset.map(
    lambda example: {
        "temp_text": f"{example['title']} {example['section']} {example['text']}".replace("\n", " ").replace("'", "")
    },
    num_proc=2,
)["temp_text"]

wiki_df = pd.DataFrame({"text": modified_texts})

Map (num_proc=2):   0%|          | 0/2101279 [00:00<?, ? examples/s]

In [4]:
modified_texts[0]

'Russell Epstein Russell Epstein Russell Epstein is a professor of psychology at the University of Pennsylvania, who studies neural mechanisms underlying visual scene perception, event perception, object recognition, and spatial navigation in humans. His lab studies the role of the Parahippocampal and retrosplenial cortices in determining how people orient themselves relative to their surroundings.'

In [15]:
%%time
file_path = ":memory:"

if os.path.exists(file_path):
    # ファイルを削除
    os.remove(file_path)
db = sqlite3.connect(file_path)
cur = db.cursor()
cur.execute('create virtual table imdb using fts5(text, tokenize="porter unicode61");')  # UNINDEXED

CPU times: user 2.3 ms, sys: 254 µs, total: 2.55 ms
Wall time: 1.23 ms


In [18]:
%%time
cur.executemany(
    "insert into imdb (text) values (?);",
    wiki_df[["text"]].to_records(index=False),
)
db.commit()

CPU times: user 1min 57s, sys: 1.32 s, total: 1min 58s
Wall time: 1min 58s


In [19]:
index = 0
text = (
    df.iloc[index].prompt
    + " "
    + df.iloc[index].A
    + " "
    + df.iloc[index].B
    + " "
    + df.iloc[index].C
    + " "
    + df.iloc[index].D
    + " "
    + df.iloc[index].E
)
text = re.sub(r"[^a-zA-Z0-9]", " ", text)
filtered_words = [word for word in text.split() if word.lower() not in stop_words]

In [21]:
%%time

q = " OR ".join(filtered_words)

res = cur.execute(
    f"""select text, rank
                      from imdb
                      where text MATCH "{q}"
                      ORDER BY rank
                      limit 1""",
).fetchall()

print(text)
print()
print(q)

res

Which of the following statements accurately describes the impact of Modified Newtonian Dynamics  MOND  on the observed  missing baryonic mass  discrepancy in galaxy clusters  MOND is a theory that reduces the observed missing baryonic mass in galaxy clusters by postulating the existence of a new form of matter called  fuzzy dark matter   MOND is a theory that increases the discrepancy between the observed missing baryonic mass in galaxy clusters and the measured velocity dispersions from a factor of around 10 to a factor of about 20  MOND is a theory that explains the missing baryonic mass in galaxy clusters that was previously considered dark matter by demonstrating that the mass is in the form of neutrinos and axions  MOND is a theory that reduces the discrepancy between the observed missing baryonic mass in galaxy clusters and the measured velocity dispersions from a factor of around 10 to a factor of about 2  MOND is a theory that eliminates the observed missing baryonic mass in g

[('Modified Newtonian dynamics Modified Newtonian dynamics MOND is an example of a class of theories known as modified gravity, and is an alternative to the hypothesis that the dynamics of galaxies are determined by massive, invisible dark matter halos. Since Milgroms original proposal, proponents of MOND have claimed to successfully predict a variety of galactic phenomena that they state are difficult to understand as consequences of dark matter.Though MOND explains the anomalously great rotational velocities of galaxies at their perimeters, it does not fully explain the velocity dispersions of individual galaxies within galaxy clusters. MOND reduces the discrepancy between the velocity dispersions and clusters observed missing baryonic mass from a factor of around 10 to a factor of about 2. However, the residual discrepancy cannot be accounted for by MOND, requiring that other explanations close the gap such as the presence of as-yet undetected missing baryonic matter.The accurate me

Int64Index([  32,   39,   65,   68,   96,  120,  124,  135,  147,  178,
            ...
            1148, 1157, 1159, 1165, 1180, 1182, 1183, 1185, 1188, 1194],
           dtype='int64', length=202)

In [50]:
df = pd.read_parquet("../input/llm-science-wikipedia-data-b/b.parquet")
df

,id,title,text
0,59737009,Biodiversity Monitoring Switzerland,The Biodiversity Monitoring Switzerland (BDM) ...
1,59737081,Bigoudène,"In Breton tradition, a coiffe bigoudène is a w..."
2,59737128,Barnabas Bala,Barnabas Yusuf Bala (20 December 1956 – 11 Jul...
3,59739888,Bagdah,Bagdah or Bagdaha may refer to: Bagdah (commun...
4,59739916,Big John Hamilton,Big John Hamilton may refer to: Big John Hamil...
...,...,...,...
351878,30045543,Bill Reeder,"William Edgar Reeder (February 20, 1922 – Marc..."
351879,3004563,"Black River Public School (Holland, Michigan)",Black River Public School is a public college ...
351880,30045661,Breg pri Konjicah,Breg pri Konjicah (pronounced [ˈbɾeːk pɾi kɔˈn...
351881,30045729,Brezje pri Ločah,Brezje pri Ločah (pronounced [ˈbreːzjɛ pɾi ˈlo...


In [162]:
%%time
import os

file_path = "../dataset/llm-science-fts/fts.db"
db = sqlite3.connect(file_path)
cur = db.cursor()
res = cur.execute(
    f"""INSERT INTO imdb(imdb) VALUES('optimize');""",
)
db.commit()

CPU times: user 3min 34s, sys: 1min 29s, total: 5min 3s
Wall time: 6min 21s


In [53]:
%%time
cur.executemany(
    "insert into imdb (text) values (?,?);",
    df[["text", "title"]].to_records(index=False),
)
db.commit()

CPU times: user 3min 57s, sys: 25.3 s, total: 4min 23s
Wall time: 4min 26s


In [11]:
import re

from nltk.corpus import stopwords

stop_words = set(stopwords.words("english"))  # 使用言語に応じて変更

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [ ]:
file_path = "tutorial.db"
db = sqlite3.connect(file_path)
cur = db.cursor()

In [35]:
res = cur.execute(
    f"""CREATE VIRTUAL TABLE ft1_r USING fts5vocab('imdb', 'row');""",
)
db.commit()

In [57]:
%%time


index = 0
text = (
    df.iloc[index].prompt
    + " "
    + df.iloc[index].A
    + " "
    + df.iloc[index].B
    + " "
    + df.iloc[index].C
    + " "
    + df.iloc[index].D
    + " "
    + df.iloc[index].E
)
text = re.sub(r"[^a-zA-Z0-9]", " ", text)
filtered_words = [word for word in text.split() if word.lower() not in stop_words]


res = cur.execute(
    f"""SELECT * FROM ft1_v WHERE term IN ({','.join(['?']*len(filtered_words))}) AND cnt < 1000 ORDER BY cnt""",
    filtered_words,
).fetchall()

res

CPU times: user 181 ms, sys: 0 ns, total: 181 ms
Wall time: 179 ms


[('called', 'text', 1, 1),
 ('considered', 'text', 1, 1),
 ('mathematical', 'text', 4, 6),
 ('around', 'title', 285, 285)]

In [58]:
%%time

q = " OR ".join([row[0] for row in res])

res = cur.execute(
    f"""select text, rank
                      from imdb
                      where text MATCH "{q }"
                      ORDER BY rank
                      limit 5""",
).fetchall()

print(text)
print()
print(q)

res

Which of the following statements accurately describes the impact of Modified Newtonian Dynamics  MOND  on the observed  missing baryonic mass  discrepancy in galaxy clusters  MOND is a theory that reduces the observed missing baryonic mass in galaxy clusters by postulating the existence of a new form of matter called  fuzzy dark matter   MOND is a theory that increases the discrepancy between the observed missing baryonic mass in galaxy clusters and the measured velocity dispersions from a factor of around 10 to a factor of about 20  MOND is a theory that explains the missing baryonic mass in galaxy clusters that was previously considered dark matter by demonstrating that the mass is in the form of neutrinos and axions  MOND is a theory that reduces the discrepancy between the observed missing baryonic mass in galaxy clusters and the measured velocity dispersions from a factor of around 10 to a factor of about 2  MOND is a theory that eliminates the observed missing baryonic mass in g

[("European Girls' Mathematical Olympiad > The European Girls' Mathematical Olympiad (EGMO) is a mathematical olympiad for girls which started in 2012, and is held in April each year. It was inspired by the China Girls Mathematical Olympiad (CGMO). Although the competition is held in Europe, it is open to female participants from all over the world, and is considered the most prestigious mathematics competition for girls. In recent years, participants from around 55 countries have been invited to the competition.",
  -17.294626741001327),
 ('Bhāskara II > Bhāskara and his works represent a significant contribution to mathematical and astronomical knowledge in the 12th century. He has been called the greatest mathematician of medieval India. His main work Siddhānta-Śiromaṇi, (Sanskrit for Crown of Treatises) is divided into four parts called Līlāvatī, Bījagaṇita, Grahagaṇita and Golādhyāya, which are also sometimes considered four independent works. These four sections deal with arithme

In [82]:
index = 6
df.iloc[index].prompt + df.iloc[index].A

'Which of the following statements accurately describes the dimension of an object in a CW complex?The dimension of an object in a CW complex is the largest n for which the n-skeleton is nontrivial, where the empty set is considered to have dimension -1 and the boundary of a discrete set of points is the empty set.'

In [84]:
df.iloc[index]

id                                                          6
prompt      Which of the following statements accurately d...
A           The dimension of an object in a CW complex is ...
B           The dimension of an object in a CW complex is ...
C           The dimension of an object in a CW complex is ...
D           The dimension of an object in a CW complex is ...
E           The dimension of an object in a CW complex dep...
answer                                                      A
context     Dimension > Moreover, the boundary of a discre...
sim_max                                              0.227938
sim_min                                               0.12335
sim_mean                                             0.203713
sim_std                                              0.027361
sim_num                                                    11
preds                                                       A
Name: 6, dtype: object